In [28]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from datetime import datetime

# Set display options for better output readability
pd.set_option('display.float_format', lambda x: '%.2f' % x)
pd.set_option('display.max_columns', None)

# Load the datasets
print("Loading datasets...")
customers = pd.read_csv("Customers.csv")
products = pd.read_csv("Products.csv")
transactions = pd.read_csv("Transactions.csv")

Loading datasets...


In [29]:
# Data Cleaning and Preprocessing
print("\nCleaning and preprocessing data...")
# Convert date columns
customers['SignupDate'] = pd.to_datetime(customers['SignupDate'])
transactions['TransactionDate'] = pd.to_datetime(transactions['TransactionDate'])

# Check for missing values
def check_missing_values(df, name):
    missing = df.isnull().sum()
    if missing.sum() > 0:
        print(f"\nMissing values in {name}:")
        print(missing[missing > 0])
    return missing.sum()

total_missing = (check_missing_values(customers, 'Customers') +
                check_missing_values(products, 'Products') +
                check_missing_values(transactions, 'Transactions'))

print(f"\nTotal missing values across all datasets: {total_missing}")


Cleaning and preprocessing data...

Total missing values across all datasets: 0


###  1. Customer Analysis


In [30]:
# 1. Customer Analysis
print("\nPerforming Customer Analysis...")

# Regional Distribution
fig_region = px.pie(customers, 
                    names='Region', 
                    title='Customer Distribution by Region',
                    color_discrete_sequence=px.colors.qualitative.Set3,
                    hole=0.3)
fig_region.update_traces(textposition='inside', textinfo='percent+label')
fig_region.update_layout(
    title_x=0.5,
    title_font_size=20,
    showlegend=True,
    legend_title="Regions",
    width=800,
    height=600
)


Performing Customer Analysis...


In [31]:
# Customer Signup Trends
customers['SignupMonth'] = customers['SignupDate'].dt.to_period('M')
monthly_signups = customers.groupby('SignupMonth').size().reset_index(name='count')
monthly_signups['SignupMonth'] = monthly_signups['SignupMonth'].astype(str)

fig_signups = px.line(monthly_signups,
                      x='SignupMonth',
                      y='count',
                      title='Customer Signup Trends Over Time',
                      markers=True)
fig_signups.update_layout(
    title_x=0.5,
    title_font_size=20,
    xaxis_title="Month",
    yaxis_title="Number of New Signups",
    xaxis_tickangle=-45,
    width=1200,
    height=600,
    showlegend=False
)

In [32]:
# 2. Product Analysis
print("\nPerforming Product Analysis...")

# Product Category Distribution
# Using groupby with tuple to avoid the warning
category_dist = (products.groupby(['Category'])  # Pass as list to avoid warning
                .agg({'ProductID': 'count'})
                .reset_index()
                .rename(columns={'ProductID': 'count'}))

# Create figure using go.Figure instead of px
fig_category = go.Figure()

fig_category.add_trace(go.Bar(
    x=category_dist['Category'],
    y=category_dist['count'],
    marker_color='rgb(158,202,225)',
    text=category_dist['count'],
    textposition='auto',
))

fig_category.update_layout(
    title={
        'text': 'Product Distribution by Category',
        'x': 0.5,
        'y': 0.95,
        'xanchor': 'center',
        'yanchor': 'top',
        'font': {'size': 20}
    },
    xaxis_title="Category",
    yaxis_title="Number of Products",
    showlegend=False,
    width=800,
    height=600,
    template='plotly_white'
)

# Price Distribution by Category using box plot
fig_price = go.Figure()

for category in products['Category'].unique():
    fig_price.add_trace(go.Box(
        y=products[products['Category'] == category]['Price'],
        name=category,
        marker_color='rgb(158,202,225)'
    ))

fig_price.update_layout(
    title={
        'text': 'Price Distribution by Category',
        'x': 0.5,
        'y': 0.95,
        'xanchor': 'center',
        'yanchor': 'top',
        'font': {'size': 20}
    },
    yaxis_title="Price (USD)",
    showlegend=False,
    width=800,
    height=600,
    template='plotly_white'
)

# Display figures
fig_category.show()
fig_price.show()

# Print summary statistics
print("\nCategory Distribution Summary:")
print(category_dist.to_string(index=False))

print("\nPrice Statistics by Category:")
price_stats = products.groupby(['Category'])[['Price']].agg(['mean', 'min', 'max']).round(2)
print(price_stats.to_string())


Performing Product Analysis...



Category Distribution Summary:
   Category  count
      Books     26
   Clothing     25
Electronics     26
 Home Decor     23

Price Statistics by Category:
             Price             
              mean   min    max
Category                       
Books       292.21 33.59 497.76
Clothing    263.72 18.82 481.78
Electronics 275.35 16.08 459.86
Home Decor  235.02 41.18 454.53


In [33]:
# 3. Transaction Analysis
print("\nPerforming Transaction Analysis...")

# Monthly Sales Trends
transactions['Month'] = transactions['TransactionDate'].dt.to_period('M')
monthly_sales = transactions.groupby('Month').agg({
    'TotalValue': 'sum',
    'TransactionID': 'count'
}).reset_index()
monthly_sales['Month'] = monthly_sales['Month'].astype(str)

# Create subplot with dual y-axis
fig_sales = make_subplots(specs=[[{"secondary_y": True}]])

fig_sales.add_trace(
    go.Scatter(x=monthly_sales['Month'],
               y=monthly_sales['TotalValue'],
               name="Total Sales",
               line=dict(color='blue')),
    secondary_y=False
)

fig_sales.add_trace(
    go.Scatter(x=monthly_sales['Month'],
               y=monthly_sales['TransactionID'],
               name="Number of Transactions",
               line=dict(color='red')),
    secondary_y=True
)

fig_sales.update_layout(
    title='Monthly Sales and Transaction Trends',
    title_x=0.5,
    title_font_size=20,
    xaxis_title="Month",
    xaxis_tickangle=-45,
    width=1200,
    height=600
)
fig_sales.update_yaxes(title_text="Total Sales (USD)", secondary_y=False)
fig_sales.update_yaxes(title_text="Number of Transactions", secondary_y=True)



Performing Transaction Analysis...


In [34]:
# 4. Combined Analysis
print("\nPerforming Combined Analysis...")

# Merge datasets for combined analysis
full_data = (transactions.merge(customers[['CustomerID', 'Region']], on='CustomerID')
            .merge(products[['ProductID', 'Category']], on='ProductID'))

# Revenue by Region and Category
region_category_revenue = full_data.groupby(['Region', 'Category'])['TotalValue'].sum().reset_index()

# Create heatmap
pivot_data = region_category_revenue.pivot(index='Region', 
                                         columns='Category', 
                                         values='TotalValue')

fig_heatmap = px.imshow(pivot_data,
                        labels=dict(x="Category", y="Region", color="Revenue"),
                        title="Revenue Heatmap: Region vs Category",
                        color_continuous_scale="Viridis",
                        aspect="auto")
fig_heatmap.update_layout(
    title_x=0.5,
    title_font_size=20,
    width=1000,
    height=600
)


Performing Combined Analysis...


In [35]:
# 5. Key Metrics Dashboard
print("\nCalculating Key Metrics...")

# Calculate key metrics
total_revenue = transactions['TotalValue'].sum()
avg_transaction = transactions['TotalValue'].mean()
total_customers = len(customers)
total_products = len(products)
total_transactions = len(transactions)

# Create metrics dashboard
fig_metrics = go.Figure()

metrics = [
    {'value': f"${total_revenue:,.2f}", 'name': "Total Revenue"},
    {'value': f"${avg_transaction:,.2f}", 'name': "Avg Transaction"},
    {'value': f"{total_customers:,}", 'name': "Total Customers"},
    {'value': f"{total_products:,}", 'name': "Total Products"},
    {'value': f"{total_transactions:,}", 'name': "Total Transactions"}
]

fig_metrics.add_trace(go.Indicator(
    mode = "number",
    value = total_revenue,
    number = {'prefix': "$", 'valueformat': ",.2f"},
    title = {'text': "Total Revenue"},
    domain = {'row': 0, 'column': 0}
))

fig_metrics.update_layout(
    grid = {'rows': 1, 'columns': 1},
    title = "Key Business Metrics",
    title_x=0.5,
    title_font_size=20,
    width=800,
    height=400
)

# Display all visualizations
print("\nDisplaying visualizations...")
fig_region.show()
fig_signups.show()
fig_category.show()
fig_price.show()
fig_sales.show()
fig_heatmap.show()
fig_metrics.show()

# Print detailed insights
print("\nKey Business Insights:")
print("-" * 50)
print(f"1. Total Revenue: ${total_revenue:,.2f}")
print(f"2. Average Transaction Value: ${avg_transaction:,.2f}")
print(f"3. Total Customers: {total_customers:,}")
print(f"4. Total Products: {total_products:,}")
print(f"5. Total Transactions: {total_transactions:,}")

# Top performing categories
top_categories = full_data.groupby('Category')['TotalValue'].sum().sort_values(ascending=False)
print("\nTop Performing Categories:")
print(top_categories.head().to_string())

# Top performing regions
top_regions = full_data.groupby('Region')['TotalValue'].sum().sort_values(ascending=False)
print("\nTop Performing Regions:")
print(top_regions.head().to_string())

# Save analysis results
print("\nSaving analysis results...")
analysis_results = pd.DataFrame({
    'Metric': ['Total Revenue', 'Avg Transaction', 'Total Customers', 
               'Total Products', 'Total Transactions'],
    'Value': [total_revenue, avg_transaction, total_customers, 
              total_products, total_transactions]
})
analysis_results.to_csv('analysis_results.csv', index=False)

print("\nAnalysis complete! Results saved to 'analysis_results.csv'")


Calculating Key Metrics...

Displaying visualizations...



Key Business Insights:
--------------------------------------------------
1. Total Revenue: $689,995.56
2. Average Transaction Value: $690.00
3. Total Customers: 200
4. Total Products: 100
5. Total Transactions: 1,000

Top Performing Categories:
Category
Books         192147.47
Electronics   180783.50
Clothing      166170.66
Home Decor    150893.93

Top Performing Regions:
Region
South America   219352.56
Europe          166254.63
North America   152313.40
Asia            152074.97

Saving analysis results...

Analysis complete! Results saved to 'analysis_results.csv'


In [ ]:
# Customer Signup Trends
customers['SignupMonth'] = customers['SignupDate'].dt.to_period('M')
monthly_signups = customers.groupby('SignupMonth').size().reset_index(name='count')
monthly_signups['SignupMonth'] = monthly_signups['SignupMonth'].astype(str)

fig_signups = px.line(monthly_signups,
                      x='SignupMonth',
                      y='count',
                      title='Customer Signup Trends Over Time',
                      markers=True)
fig_signups.update_layout(
    title_x=0.5,
    title_font_size=20,
    xaxis_title="Month",
    yaxis_title="Number of New Signups",
    xaxis_tickangle=-45,
    width=1200,
    height=600,
    showlegend=False
)

# Business Insights Report

## Key Business Insights
The analysis reveals significant trends and opportunities to optimize business strategies:

1. **Regional Trends**  
   Customers predominantly hail from **South America**, contributing **$219,352.56** to revenue.  
   👉 Suggestion: Focus marketing efforts and promotional activities in this region for maximum impact.

2. **Top-Selling Categories**  
   The highest revenue-generating categories are:  
   - **Books**: $192,147.47  
   - **Electronics**: $180,783.50  
   - **Clothing**: $166,170.66  
   👉 Suggestion: Prioritize these categories for promotions and inventory management.

3. **Sales Patterns**  
   Seasonal peaks, such as increased activity during **July**, are evident.  
   👉 Suggestion: Leverage these patterns for strategic marketing and inventory planning.

4. **Customer Growth Trends**  
   Further analysis of customer signups could reveal trends, such as consistent growth or spikes during specific periods.  
   👉 Suggestion: Use these trends to enhance customer engagement strategies.

5. **High Revenue Sources**  
   A combination of **South American customers** and categories like **Books** and **Electronics** drives significant revenue.  
   👉 Suggestion: Focus on these segments for targeted business development.

---

## Top Performing Categories
| Category      | Revenue       |
|---------------|---------------|
| **Books**     | $192,147.47  |
| **Electronics** | $180,783.50 |
| **Clothing**  | $166,170.66  |
| **Home Decor** | $150,893.93 |

---

## Top Performing Regions
| Region          | Revenue       |
|------------------|---------------|
| **South America** | $219,352.56 |
| **Europe**        | $166,254.63 |
| **North America** | $152,313.40 |
| **Asia**          | $152,074.97 |

---

## Next Steps
The results highlight clear opportunities to refine marketing strategies, tailor regional promotions, and manage inventory effectively. Further exploration of customer trends and seasonal patterns can unlock additional insights for sustained growth.

**Analysis saved to:** `analysis_results.csv`
